# Mapping cultural remains with object detection
All forests in Sweden, both managed forests and natural old-growth forests in national parks, contain a cultural
heritage. The long history of forest utilization in Sweden has left a rich legacy of diverse types of ancient
monuments and other kinds of cultural remains that document our relationship with the forest and its importance
for Sweden’s development. However, the cultural heritage is too often damaged in forestry operations. The aim of
the project is to do research and develop operationally useful maps that can be used
to identify, protect and enhance the cultural remains in Swedish forests, thereby reducing the destruction of
cultural heritage in our forest landscapes.

In [3]:
!pip install rtree
!pip install torch
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.4 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


## Topographical modeling

### Select Laz tiles intersecting field data

In [2]:
!python /workspace/code/create_aoi_poolygon.py /workspace/lidar/none.shp /workspace/data/hunting_pits/Fangstgrop_training_Holmen_Cissi_695st_220214.shp /workspace/lidar/pooled_laz_files/ /workspace/data/hunting_pits/laz/ 

/workspace/code/create_aoi_poolygon.py:37: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  main(**args)
Traceback (most recent call last):
  File "/workspace/code/create_aoi_poolygon.py", line 37, in <module>
    main(**args)
  File "/workspace/code/create_aoi_poolygon.py", line 25, in main
    copy_tiles(footprint, field_data, input_directory, output_directory)
  File "/workspace/code/create_aoi_poolygon.py", line 13, in copy_tiles
    intersect = gpd.sjoin(lidar_tiles_footprint, field, how='inner', op='intersects')
  File "/usr/local/lib/python3.8/dist-packages/geopandas/tools/sjoin.py", line 124, in sjoin
    indices = _geom_predicate_query(left_df, right_df, predicate)
  File "/usr/local/lib/python3.8/dist-packages/geopandas/tools/sjoin.py", line 216, in _geom_predicate_query
    sindex = right_df.sindex
  File "/usr/local/lib/python3.8/dist-packages/geopandas/base.py", line 2637, in sindex
   

### Convert selected laz files to DEM

In [ ]:
!python /workspace/code/laz_to_dem.py /workspace/data/hunting_pits/laz/ /workspace/data/hunting_pits/dem_tiles/

### Extract topographical indices

In [ ]:
!python /workspace/code/Extract_topographcical_indices.py /workspace/temp/ /workspace/data/hunting_pits/laz/ /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/ /workspace/data/hunting_pits/topographical_indices_normalized/slope/ /workspace/data/hunting_pits/topographical_indices_normalized/hpmf/ /workspace/data/hunting_pits/topographical_indices_normalized/stdon/

## Labels

**Hunting pits integer masks**

In [ ]:
!python /workspace/code/create_labels.py /workspace/data/hunting_pits/dem_tiles/ /workspace/data/hunting_pits/hunting_pits.shp /workspace/data/hunting_pits/object_detection_data/label_tiles/

All data were split into image chips with the size 256x256. Note that the directories needs to be empty before running the split script. I found it esiest to recreate the directories to avoid errors.

In [7]:
# Start by clearing directories of existing data
import os

#shutil.rmtree('/workspace/data/split_data/') #this fails alot to it manually
os.mkdir('/workspace/data/hunting_pits/split_data/')
os.mkdir('/workspace/data/hunting_pits/split_data/labels/')
os.mkdir('/workspace/data/hunting_pits/split_data/slope/')
os.mkdir('/workspace/data/hunting_pits/split_data/hillshade/')
os.mkdir('/workspace/data/hunting_pits/split_data/hpmf/')
os.mkdir('/workspace/data/hunting_pits/split_data/stdon/')  

# Split data
# Hillshade 
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/ /workspace/data/hunting_pits/split_data/hillshade/ --tile_size 256
# Slope
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/topographical_indices_normalized/slope/ /workspace/data/hunting_pits/split_data/slope/ --tile_size 256
# High pass median filter
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/topographical_indices_normalized/hpmf/ /workspace/data/hunting_pits/split_data/hpmf/ --tile_size 256
# High pass median filter
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/topographical_indices_normalized/stdon/ /workspace/data/hunting_pits/split_data/stdon/ --tile_size 256
# Labels
!python /workspace/code/split_training_data.py /workspace/data/hunting_pits/label_tiles/ /workspace/data/hunting_pits/split_data/labels/ --tile_size 256

Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 0 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 1
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 494.13img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/18D022_67450_5775_25.tif are added at /workspace/data/hunting_pits/split_data/hillshade/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 400 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 401
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 497.43img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/18D022_67475_5750_25.tif are added at /workspace/data/hunting_pits/split_data/hillshade/
In

Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 6400 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 6401
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 493.86img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/19F047_71075_7200_25.tif are added at /workspace/data/hunting_pits/split_data/hillshade/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 6800 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 6801
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 491.96img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/19F047_71075_7225_25.tif are added at /workspace/data/hunting_pits/split_data/hills

Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 12800 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 12801
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 495.85img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/19G013_71525_7025_25.tif are added at /workspace/data/hunting_pits/split_data/hillshade/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 13200 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 13201
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 513.56img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/19G013_71550_7000_25.tif are added at /workspace/data/hunting_pits/split_data/h

Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 19200 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 19201
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 505.18img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/19G013_71725_7200_25.tif are added at /workspace/data/hunting_pits/split_data/hillshade/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 19600 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 19601
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 497.59img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/19G013_71725_7225_25.tif are added at /workspace/data/hunting_pits/split_data/h

Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 25600 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 25601
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 489.84img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/20E018_68825_4800_25.tif are added at /workspace/data/hunting_pits/split_data/hillshade/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 26000 files in the /workspace/data/hunting_pits/split_data/hillshade/
New image name will start with 26001
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 487.88img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/hillshade/20E018_68850_4800_25.tif are added at /workspace/data/hunting_pits/split_data/h

...

Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 492.74img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/stdon/20E018_68800_4775_25.tif are added at /workspace/data/hunting_pits/split_data/stdon/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 25200 files in the /workspace/data/hunting_pits/split_data/stdon/
New image name will start with 25201
Generating: 100%|███████████████████████████| 400/400 [00:00<00:00, 496.33img/s]
400 tiles sample of /workspace/data/hunting_pits/topographical_indices_normalized/stdon/20E018_68800_4800_25.tif are added at /workspace/data/hunting_pits/split_data/stdon/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 25600 files in the /workspace/data/hunting_pits/split_data/stdon/
New image name will start with 25601
Generating: 10

Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 332.74img/s]
400 tiles sample of /workspace/data/hunting_pits/label_tiles/19F047_71025_7100_25.tif are added at /workspace/data/hunting_pits/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 5600 files in the /workspace/data/hunting_pits/split_data/labels/
New image name will start with 5601
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 335.56img/s]
400 tiles sample of /workspace/data/hunting_pits/label_tiles/19F047_71025_7225_25.tif are added at /workspace/data/hunting_pits/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 6000 files in the /workspace/data/hunting_pits/split_data/labels/
New image name will start with 6001
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 

Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 339.81img/s]
400 tiles sample of /workspace/data/hunting_pits/label_tiles/19G013_71500_7025_25.tif are added at /workspace/data/hunting_pits/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 12400 files in the /workspace/data/hunting_pits/split_data/labels/
New image name will start with 12401
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 332.81img/s]
400 tiles sample of /workspace/data/hunting_pits/label_tiles/19G013_71525_7000_25.tif are added at /workspace/data/hunting_pits/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 12800 files in the /workspace/data/hunting_pits/split_data/labels/
New image name will start with 12801
Generating: 100%|███████████████████████████| 400/400 [00:01<00:

Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 336.71img/s]
400 tiles sample of /workspace/data/hunting_pits/label_tiles/19G013_71725_7175_25.tif are added at /workspace/data/hunting_pits/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 19200 files in the /workspace/data/hunting_pits/split_data/labels/
New image name will start with 19201
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 332.60img/s]
400 tiles sample of /workspace/data/hunting_pits/label_tiles/19G013_71725_7200_25.tif are added at /workspace/data/hunting_pits/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 19600 files in the /workspace/data/hunting_pits/split_data/labels/
New image name will start with 19601
Generating: 100%|███████████████████████████| 400/400 [00:01<00:

Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 323.82img/s]
400 tiles sample of /workspace/data/hunting_pits/label_tiles/20E018_68825_4800_25.tif are added at /workspace/data/hunting_pits/split_data/labels/
Input Image File Shape (D, H, W):(1, 5000, 5000)
crop_size=256, stride=256
Padding Image File Shape (D, H, W):(1, 5120, 5120)
There are 26000 files in the /workspace/data/hunting_pits/split_data/labels/
New image name will start with 26001
Generating: 100%|███████████████████████████| 400/400 [00:01<00:00, 329.60img/s]
400 tiles sample of /workspace/data/hunting_pits/label_tiles/20E018_68850_4800_25.tif are added at /workspace/data/hunting_pits/split_data/labels/


Not all of the splited image chips contained any objects. These chips were removed.

In [10]:
!python /workspace/code/remove_unlabled_chips.py 1 /workspace/data/hunting_pits/split_data/labels/ /workspace/data/hunting_pits/split_data/hillshade/ /workspace/data/hunting_pits/split_data/slope/ /workspace/data/hunting_pits/split_data/hpmf/ /workspace/data/hunting_pits/split_data/stdon/

**Hunting pits bounding boxes**\
The segmentation masks were converted to yolo labels for object detection

In [9]:
!python /workspace/code/masks_to_boxes.py /workspace/temp/ /workspace/data/hunting_pits/split_data/labels/ /workspace/data/hunting_pits/object_detection_data/box_labels/ 

tensor([[ 38., 176.,  69., 232.]])
tensor([[148., 234., 165., 251.]])
tensor([[ 82.,   1., 253.,  85.]])
tensor([[ 32.,  13., 167., 216.]])
tensor([[ 66.,  31., 148., 148.]])
tensor([[178.,  51., 192.,  64.]])
tensor([[100., 214., 113., 227.]])
tensor([[100.,  24., 113.,  37.]])
tensor([[138., 208., 175., 255.]])
tensor([[ 96.,   0., 160., 255.]])
tensor([[145.,   0., 241., 112.]])
tensor([[111., 136., 197., 164.]])
tensor([[ 28.,  27.,  51., 221.]])
tensor([[233., 133., 251., 152.]])
tensor([[ 81., 210.,  98., 227.]])
tensor([[  4.,  21.,  29., 240.]])
tensor([[247., 242., 255., 255.]])
tensor([[  0., 190., 190., 255.]])
tensor([[ 62., 200., 199., 222.]])
tensor([[228.,  33., 252., 134.]])
tensor([[133.,  34., 255.,  61.]])
tensor([[  0.,   0., 255.,  60.]])
tensor([[0., 0., 8., 4.]])
tensor([[140.,  36., 159.,  55.]])
tensor([[ 67., 109., 176., 247.]])
tensor([[177.,  17., 199.,  39.]])
tensor([[112.,  99., 126., 113.]])
tensor([[ 46.,  84.,  77., 103.]])
tensor([[236., 151., 253., 1

tensor([[ 32., 109., 238., 147.]])
tensor([[155., 232., 188., 255.]])
tensor([[ 17., 225.,  51., 255.]])
tensor([[132., 228., 255., 255.]])
tensor([[  0., 176., 241., 255.]])
tensor([[ 10., 137.,  43., 171.]])
tensor([[ 81., 165., 169., 252.]])
tensor([[ 15.,  91., 200., 204.]])
tensor([[ 77.,   0., 187., 143.]])
tensor([[ 26.,   0., 193.,  49.]])
tensor([[117.,   0., 255.,  95.]])
tensor([[  0.,   0., 247., 166.]])
tensor([[ 51.,  62., 255., 255.]])
tensor([[ 0., 58., 26., 92.]])
tensor([[52.,  0., 83., 11.]])
tensor([[ 79.,  97., 254., 243.]])
tensor([[  5.,  93.,  38., 126.]])
tensor([[60., 31., 93., 64.]])
tensor([[ 27.,  74.,  61., 107.]])
tensor([[249., 179., 255., 206.]])
tensor([[  0.,  37., 236., 209.]])
tensor([[ 17., 255.,  20., 255.]])
tensor([[  0., 109.,  14., 143.]])
tensor([[104., 144., 255., 238.]])
tensor([[  0., 148.,   5., 173.]])
tensor([[232., 170., 255., 203.]])
tensor([[  0., 172.,   9., 202.]])
tensor([[130.,   6., 164.,  40.]])
tensor([[168.,  39., 202.,  73.]